In [33]:
# IMPORT PACKAGES

# Import general packages
import pandas as pd
from parsel import Selector
import time
import os

# Import Selenium packages
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

# Import Beautiful Soup packages
from bs4 import BeautifulSoup

In [2]:
# IMPORT VENUES IN AMS

# Import excel file with venues in Amsterdam (from Jeroen)
venues_ams = pd.read_excel (r'/Users/LizzyDRB/Documents/MScDS/GoogleReviews/amsterdam_venues_place_id.xlsx')

# Preview of data
venues_ams

,name,street,housenumber,city,postcode,lat,lon,RD_x,RD_y,tile_code,place_id
0,De Kletskop,Zeedijk,10,Amsterdam,1012AX,52.375980,4.900340,121846.813111,487678.771043,2436_9753,ChIJ1zQxxrkJxkcRG90MVXyWSIc
1,Haven van Texel,NaN,NaN,NaN,NaN,52.375789,4.900279,121842.516201,487657.492050,2436_9753,ChIJFS4HyLkJxkcRoMP5yz4NcJ4
2,Kam Yin,Warmoesstraat,6,Amsterdam,NaN,52.376236,4.899454,121786.693292,487707.593209,2435_9754,ChIJC6hwM7gJxkcRj8RMeZoWSTA
3,De Ooievaar,Sint Olofspoort,1,Amsterdam,1012AJ,52.376284,4.899940,121819.816378,487712.766897,2436_9754,ChIJWwxw8NEJxkcREUXIGY2SHVg
4,Bitterzoet,Spuistraat,2,Amsterdam,1012TS,52.377357,4.894142,121425.813798,487834.767054,2428_9756,ChIJsY8odsgJxkcRX1QkUoMSOKk
...,...,...,...,...,...,...,...,...,...,...,...
2968,NaN,NaN,NaN,NaN,NaN,52.410843,4.921504,123312.910259,491548.242693,2466_9830,NaN
2969,NaN,NaN,NaN,NaN,NaN,52.412294,4.922164,123358.846813,491709.431973,2467_9834,NaN
2970,NaN,NaN,NaN,NaN,NaN,52.371458,4.930970,123929.340496,487162.055915,2478_9743,NaN
2971,Czaar,NaN,NaN,NaN,NaN,52.371840,4.931803,123986.342783,487204.201669,2479_9744,ChIJk67ICQwJxkcR4vod07nA2qA


In [3]:
# DROP NAN

# Change place_id's into strings
for row in range(venues_ams.shape[0]):
    venues_ams['place_id'][row] = str(venues_ams['place_id'][row])
    
# Drop rows where place_id is nan
for row in range(venues_ams.shape[0]):
    if venues_ams['place_id'][row] == 'nan':
        venues_ams = venues_ams.drop(row)
        
# Reset the index to range from 0, 1965        
venues_ams = venues_ams.reset_index()

/var/folders/rn/k7vf4gp97fb11dzjbcr7w8bc0000gn/T/ipykernel_857/3004530383.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venues_ams['place_id'][row] = str(venues_ams['place_id'][row])


In [4]:
# FUNCTIONS

# Function to get review overview from scraper
# Returns dataframe with Name, Rate, Time, Text
def review_overview(result_set, name):
    rev_dict = {'Name': name,
                'Review Rate': [],
                'Review Time': [],
                'Review Text' : []}
    for result in result_set:
        review_rate = result.find('span', class_='ODSEW-ShBeI-H1e3jb')["aria-label"]
        review_time = result.find('span',class_='ODSEW-ShBeI-RgZmSc-date').text
        review_text = result.find('span',class_='ODSEW-ShBeI-text').text
        rev_dict['Review Rate'].append(review_rate)
        rev_dict['Review Time'].append(review_time)
        rev_dict['Review Text'].append(review_text)   
    return(pd.DataFrame(rev_dict))

In [5]:
# Count rows = 1965
print('Shape of venues_ams is', venues_ams.shape)

venues_ams

Shape of venues_ams is (1965, 12)


,index,name,street,housenumber,city,postcode,lat,lon,RD_x,RD_y,tile_code,place_id
0,0,De Kletskop,Zeedijk,10,Amsterdam,1012AX,52.375980,4.900340,121846.813111,487678.771043,2436_9753,ChIJ1zQxxrkJxkcRG90MVXyWSIc
1,1,Haven van Texel,NaN,NaN,NaN,NaN,52.375789,4.900279,121842.516201,487657.492050,2436_9753,ChIJFS4HyLkJxkcRoMP5yz4NcJ4
2,2,Kam Yin,Warmoesstraat,6,Amsterdam,NaN,52.376236,4.899454,121786.693292,487707.593209,2435_9754,ChIJC6hwM7gJxkcRj8RMeZoWSTA
3,3,De Ooievaar,Sint Olofspoort,1,Amsterdam,1012AJ,52.376284,4.899940,121819.816378,487712.766897,2436_9754,ChIJWwxw8NEJxkcREUXIGY2SHVg
4,4,Bitterzoet,Spuistraat,2,Amsterdam,1012TS,52.377357,4.894142,121425.813798,487834.767054,2428_9756,ChIJsY8odsgJxkcRX1QkUoMSOKk
...,...,...,...,...,...,...,...,...,...,...,...,...
1960,2961,Pension Homeland,NaN,NaN,NaN,NaN,52.372880,4.916515,122945.963716,487326.545156,2458_9746,ChIJtdZbkaYJxkcRc9Qcc-bXUGc
1961,2962,Frank's Smoke House,Oostenburgervoorstraat,1,NaN,NaN,52.368902,4.921978,123315.154310,486881.482173,2466_9737,ChIJvy3dlqAJxkcRbg-tib1GMBU
1962,2963,Caf√© Daan & Daan,Kattenburgerplein,39,Amsterdam,1018KK,52.370886,4.916357,122933.817222,487104.766978,2458_9742,ChIJq5WAS6EJxkcReT5wzYdIZm4
1963,2971,Czaar,NaN,NaN,NaN,NaN,52.371840,4.931803,123986.342783,487204.201669,2479_9744,ChIJk67ICQwJxkcR4vod07nA2qA


In [6]:
# INITIATE ALL VARIABLES

# Create df containing all Google Maps links
cols = ['google_link']
url_list = []
 
for row in range(venues_ams.shape[0]):
    p = venues_ams['place_id'][row]
    url = 'https://www.google.com/maps/place/?q=place_id:'+p
    url_list.append([url])
 
google_links = pd.DataFrame(url_list, columns=cols)

# Append df to venues_ams
venues_ams = venues_ams.join(google_links)

In [7]:
# NEW VARIABLES FOR SAVING REVIEWS
# Create new dataframe that will contain info of the venuews
cols2 = ['name', 'place_id', 'url']
total_reviews = pd.DataFrame(columns=cols) # final df for all reviews
running_reviews2 = pd.DataFrame(columns=cols2) # for testing

In [16]:
running_reviews = pd.DataFrame()


""


In [14]:
for index, cols in venues_ams.iterrows():
    print(index, cols[1], cols[12])

0 De Kletskop https://www.google.com/maps/place/?q=place_id:ChIJ1zQxxrkJxkcRG90MVXyWSIc
1 Haven van Texel https://www.google.com/maps/place/?q=place_id:ChIJFS4HyLkJxkcRoMP5yz4NcJ4
2 Kam Yin https://www.google.com/maps/place/?q=place_id:ChIJC6hwM7gJxkcRj8RMeZoWSTA
3 De Ooievaar https://www.google.com/maps/place/?q=place_id:ChIJWwxw8NEJxkcREUXIGY2SHVg
4 Bitterzoet https://www.google.com/maps/place/?q=place_id:ChIJsY8odsgJxkcRX1QkUoMSOKk
5 New Dutch https://www.google.com/maps/place/?q=place_id:ChIJs4EH2rwJxkcRIUllSD3jWA8
6 Jennifer https://www.google.com/maps/place/?q=place_id:ChIJVXealLU_xkcRja_At0z9AGY
7 Sluyswacht https://www.google.com/maps/place/?q=place_id:ChIJzccHg74JxkcRufHICkn86wo
8 Cafe de Zon https://www.google.com/maps/place/?q=place_id:ChIJdZ5ys88JxkcRJl584hErf4Y
9 Nossa Senhora https://www.google.com/maps/place/?q=place_id:ChIJVXealLU_xkcRja_At0z9AGY
10 Maria https://www.google.com/maps/place/?q=place_id:ChIJVXealLU_xkcRja_At0z9AGY
11 Oost-West https://www.google.com/maps/p

485 Cafe Spanje https://www.google.com/maps/place/?q=place_id:ChIJi7xhMnjjQgwR7KNoB5Qs7KY
486 Bam Boa https://www.google.com/maps/place/?q=place_id:ChIJhaWQJPwJxkcR54jGw_3fFLE
487 East57 https://www.google.com/maps/place/?q=place_id:ChIJqZijS-RYwokRAAmnNJESbuY
488 The Lobby https://www.google.com/maps/place/?q=place_id:ChIJRyW66n0JxkcRAO3hGrNP9Nc
489 Vishandel Rinkoen https://www.google.com/maps/place/?q=place_id:ChIJd-jGsW0JxkcR_BOh1-S9mHM
490 Carmen's Food Corner https://www.google.com/maps/place/?q=place_id:ChIJ13gXDHIJxkcRGrSV1Yk-it4
491 McDonald's https://www.google.com/maps/place/?q=place_id:ChIJVXealLU_xkcRja_At0z9AGY
492 Mi-Gudu https://www.google.com/maps/place/?q=place_id:ChIJn_rVynMJxkcRqd8wJLJXcFg
493 Ada Restaurant (closed) https://www.google.com/maps/place/?q=place_id:ChIJsVykE07ixUcRWoQ5GQNG_HE
494 Pacific Parc https://www.google.com/maps/place/?q=place_id:ChIJ82Ku_mYJxkcRZWFK2mf5IyQ
495 Westergasterras https://www.google.com/maps/place/?q=place_id:ChIJMVYWmX_ixUcRWE9x4V

817 Huizinglaan https://www.google.com/maps/place/?q=place_id:ChIJZ63feyjixUcRUSIgC-llcMo
818 Cedars https://www.google.com/maps/place/?q=place_id:ChIJYdhQuh7ixUcRtaBnJefZQYE
819 George W.P.A. https://www.google.com/maps/place/?q=place_id:ChIJtUptyeQJxkcRF8u0aSLiTFc
820 CoffeeConcepts https://www.google.com/maps/place/?q=place_id:ChIJ2RwuuuQJxkcROVNf25dGhqQ
821 brasserie bark https://www.google.com/maps/place/?q=place_id:ChIJF-Jka_AJxkcRrMQC7qop15g
822 Roberto's Restaurant https://www.google.com/maps/place/?q=place_id:ChIJsYZOU_kJxkcRoU8-ty7gB98
823 Kersentuin https://www.google.com/maps/place/?q=place_id:ChIJ9wKP_iR0w0cRYERjInpC0Xc
824 Lunchroom Grannies https://www.google.com/maps/place/?q=place_id:ChIJK7827hzixUcRuj4VAnlanUk
825 Mac‚Äôs Food Place https://www.google.com/maps/place/?q=place_id:ChIJ-5vd-CTixUcRCY63wJrYU3Q
826 Flavours of India https://www.google.com/maps/place/?q=place_id:ChIJ30ccGUejlVQRdtFzkfcitGY
827 Broodjeszaak Polat https://www.google.com/maps/place/?q=place_id:

985 Genroku https://www.google.com/maps/place/?q=place_id:ChIJD95TceoJxkcRMPRmhjAdGO0
986 Norling https://www.google.com/maps/place/?q=place_id:ChIJc4THduoJxkcRfuHyolwe6nY
987 Rainbowl https://www.google.com/maps/place/?q=place_id:ChIJ_RhD65EJxkcRiOH41wnKX5A
988 The Big Apple https://www.google.com/maps/place/?q=place_id:ChIJAchFdOoJxkcR7zPvafBoCjQ
989 Ginger https://www.google.com/maps/place/?q=place_id:ChIJVXealLU_xkcRja_At0z9AGY
990 Herengracht https://www.google.com/maps/place/?q=place_id:ChIJPdcBy8AJxkcRliIZlE9jFj8
991 The Seafood Bar https://www.google.com/maps/place/?q=place_id:ChIJw6roY-0JxkcR4kx59nXldmY
992 Cafe Berkhout https://www.google.com/maps/place/?q=place_id:ChIJU9ZPEO0JxkcRudfaJ9vs7Oo
993 Eazie https://www.google.com/maps/place/?q=place_id:ChIJmQGWbu0JxkcRxBknc8edw6I
994 Bouch√©e https://www.google.com/maps/place/?q=place_id:ChIJVXealLU_xkcRja_At0z9AGY
995 Thai Food Caf√© https://www.google.com/maps/place/?q=place_id:ChIJdyMN028JxkcRqJE6av5MSxA
996 Bar Wisse https://w

1340 De Pannenkoekerij https://www.google.com/maps/place/?q=place_id:ChIJRe_tAscJxkcR2phf1tCtkHk
1341 Stadspaleis https://www.google.com/maps/place/?q=place_id:ChIJJzq_PMEJxkcRYMVxY4ymZJY
1342 Miu https://www.google.com/maps/place/?q=place_id:ChIJf6GbzMYJxkcR69jgOeUbElo
1343 De Nissen https://www.google.com/maps/place/?q=place_id:ChIJVXealLU_xkcRja_At0z9AGY
1344 Theatercaf√© De Richel https://www.google.com/maps/place/?q=place_id:ChIJ89yojMAJxkcRoTUDKaPW-wQ
1345 The Lobby https://www.google.com/maps/place/?q=place_id:ChIJRyW66n0JxkcRAO3hGrNP9Nc
1346 Bagels & Beans https://www.google.com/maps/place/?q=place_id:ChIJVXealLU_xkcRja_At0z9AGY
1347 Breda https://www.google.com/maps/place/?q=place_id:ChIJR_22I8QJxkcR2hLke23mATQ
1348 Ellis Gourmet Burger https://www.google.com/maps/place/?q=place_id:ChIJ___y-8YJxkcRazeKOde9ty8
1349 La Tertulia Coffeeshop https://www.google.com/maps/place/?q=place_id:ChIJdXov18IJxkcR2ApdynzKaPs
1350 A. van Wees Proeflokaal https://www.google.com/maps/place/?q=pl

1651 Wolf Atelier https://www.google.com/maps/place/?q=place_id:ChIJ6aWA3ssJxkcRr-roXgFOyi8
1652 Hoogendam https://www.google.com/maps/place/?q=place_id:ChIJ6ff6-ssJxkcRE6xkV5nRsKA
1653 The burrito maker https://www.google.com/maps/place/?q=place_id:ChIJiQtgE9IJxkcRUBZ9zLCpFE8
1654 Twee Zwaantjes https://www.google.com/maps/place/?q=place_id:ChIJ3d9HP8UJxkcRjTZAEeg6yNE
1655 Gezondigd https://www.google.com/maps/place/?q=place_id:ChIJV-jNHTUjwUcRlvNRZ6LC108
1656 Tijger & de Vis https://www.google.com/maps/place/?q=place_id:ChIJUUlQ0s8JxkcRdPCKCf9pOFc
1657 Caf√© De Blauwe Pan https://www.google.com/maps/place/?q=place_id:ChIJY4BpdNAJxkcRY7KVAzGKi_U
1658 Fiko https://www.google.com/maps/place/?q=place_id:ChIJ3Q26nNMJxkcRggqT4TlJZfE
1659 San George https://www.google.com/maps/place/?q=place_id:ChIJVXealLU_xkcRja_At0z9AGY
1660 Koffie Academie https://www.google.com/maps/place/?q=place_id:ChIJid7DpeYJxkcRlG7Et4uem_M
1661 La Polpetta https://www.google.com/maps/place/?q=place_id:ChIJkRWm-g_ix

In [9]:
# START THE CHROME DRIVER

# Open Chromedriver
chromedrive_path = '/Users/LizzyDRB/Documents/MScDS/GoogleReviews/chromedriver' # use the path to the driver you downloaded from previous steps
driver = webdriver.Chrome(chromedrive_path)

# Open Google Maps
url='https://www.google.com/maps/place/'
driver.get(url)

# Accept cookies
wait = WebDriverWait(driver, 10)

button_cookies = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[4]/form/div/div/button/span')))
driver.execute_script("arguments[0].click()", button_cookies)

/var/folders/rn/k7vf4gp97fb11dzjbcr7w8bc0000gn/T/ipykernel_857/1087606482.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedrive_path)


In [10]:
# Loop over all Google Maps venue links
for rows, cols in venues_ams.iterrows():
    
    # Skip venues 0-17, already extracted
    if rows < 18:
        continue
    
    # Skip Kam Yin (it produces problems)
    #if cols[1] == "Kam Yin":
    #    continue
    
    # Continue to next venue if reviews already extracted
    if os.path.exists("reviews_"+str(cols[1])+"_"+str(cols[11])+".csv"):
        continue
    
    # Open Chromedriver with Google Maps url
    url=cols[12]
    driver.get(url)
    
    # Find name of venue
    driver.implicitly_wait(5)
    find_name = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1/span[1]').text
    
    # If Amsterdam Google Maps page, continue to next venue
    if find_name == "Amsterdam":
        continue
    
    # Go to All Reviews page
    wait = WebDriverWait(driver, 10)

    button_reviews = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/span[1]/span/span[1]/span[2]/span[1]/button')))
    driver.execute_script("arguments[0].click()", button_reviews)

    # Parse Reviews
    page_content = driver.page_source
    
    # Parse content from HTML page
    response = Selector(page_content)

    # Iterate over reviews
    results = []

    for el in response.xpath('//div/div[@data-review-id]/div[contains(@class, "content")]'):
        results.append({
            #'title': el.xpath('.//div[contains(@class, "title")]/span/text()').extract_first(''),
            'rating': el.xpath('.//span[contains(@aria-label, "stars")]/@aria-label').extract_first('').replace('stars' ,'').strip(),
            'body': el.xpath('.//span[contains(@class, "text")]/text()').extract_first(''),
        })

    #Find the total number of reviews
    driver.implicitly_wait(5)
    total_number_of_reviews = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[2]/div/div[2]/div[2]').text.split(" ")[0]


    if '.' in total_number_of_reviews:
        total_number_of_reviews = int(total_number_of_reviews.replace('.',''))
    else:
        int(total_number_of_reviews)

    #Find scroll layout
    scrollable_div = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')

    #Scroll as many times as necessary to load all reviews
    for i in range(0,(round(int(total_number_of_reviews)/10 - 1))):
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        time.sleep(10)

    # Parse Reviews with Beautiful Soup
    response = BeautifulSoup(driver.page_source, 'html.parser')
    reviews = response.find_all('div', class_='ODSEW-ShBeI NIyLF-haAclf gm2-body-2')

    
    # Put reviews in dataframe
    df_reviews = review_overview(reviews, cols[1])
    
    df_reviews.to_csv("reviews_"+str(cols[1])+"_"+str(cols[11])+".csv")

/var/folders/rn/k7vf4gp97fb11dzjbcr7w8bc0000gn/T/ipykernel_857/2769499606.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  find_name = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1/span[1]').text
/var/folders/rn/k7vf4gp97fb11dzjbcr7w8bc0000gn/T/ipykernel_857/2769499606.py:40: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  total_number_of_reviews = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[2]/div/div[2]/div[2]').text.split(" ")[0]
/var/folders/rn/k7vf4gp97fb11dzjbcr7w8bc0000gn/T/ipykernel_857/2769499606.py:49: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  scrollable_div = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')


KeyboardInterrupt: 

In [17]:
running_reviews2

,name,place_id,url,Name,Review Rate,Review Time,Review Text
0,NaN,NaN,NaN,De Kletskop,5 stars,4 months ago,Great 'bruin café' in Amsterdam that I discove...
1,NaN,NaN,NaN,De Kletskop,5 stars,2 years ago,"Súper cool people, the bartender was super nic..."
2,NaN,NaN,NaN,De Kletskop,5 stars,a month ago,Friendly staff.
3,NaN,NaN,NaN,De Kletskop,5 stars,2 years ago,Had a great evening in this cafe in the centre...
4,NaN,NaN,NaN,De Kletskop,5 stars,6 years ago,Cosy local pub near the central station and Am...
...,...,...,...,...,...,...,...
235,NaN,NaN,NaN,Porto Fino,5 stars,2 years ago,(Translated by Google) + nice staff\n+ good fo...
236,NaN,NaN,NaN,Porto Fino,3 stars,2 years ago,(Translated by Google) Pizzas are small and no...
237,NaN,NaN,NaN,Porto Fino,1 star,3 years ago,(Translated by Google) Shit of thieves place o...
238,NaN,NaN,NaN,Porto Fino,5 stars,3 years ago,(Translated by Google) Good pizzas and the hos...


In [12]:
# Save to excel (De Kletskop t/m Porto Fino)
running_reviews2.to_excel("reviews_0-17.xlsx")

In [13]:
# Save to csv (De Kletskop t/m Porto Fino)
running_reviews2.to_csv("reviews_0-17.csv")

In [28]:
# Save all venues to csv
running_reviews2.iloc[0:80].to_csv("reviews_De Kletskop_ChIJ1zQxxrkJxkcRG90MVXyWSIc.csv")


In [29]:
running_reviews2.iloc[81:420].to_csv("reviews_Haven van Texel_ChIJFS4HyLkJxkcRoMP5yz4NcJ4.csv")

In [30]:
running_reviews2.iloc[421:1440].to_csv("reviews_Bitterzoet_ChIJsY8odsgJxkcRX1QkUoMSOKk.csv")

In [40]:
for rows, cols in venues_ams.iterrows():
    if os.path.exists("reviews_"+str(cols[1])+"_"+str(cols[11])+".csv"):
        print(cols[1], cols[11], "this file exists", "reviews_"+str(cols[1])+"_"+str(cols[11])+".csv")

De Kletskop ChIJ1zQxxrkJxkcRG90MVXyWSIc this file exists reviews_De Kletskop_ChIJ1zQxxrkJxkcRG90MVXyWSIc.csv
Haven van Texel ChIJFS4HyLkJxkcRoMP5yz4NcJ4 this file exists reviews_Haven van Texel_ChIJFS4HyLkJxkcRoMP5yz4NcJ4.csv
Bitterzoet ChIJsY8odsgJxkcRX1QkUoMSOKk this file exists reviews_Bitterzoet_ChIJsY8odsgJxkcRX1QkUoMSOKk.csv


In [41]:
for rows, cols in venues_ams.iterrows():
    if rows < 18:
        print(rows, cols[1])

0 De Kletskop
1 Haven van Texel
2 Kam Yin
3 De Ooievaar
4 Bitterzoet
5 New Dutch
6 Jennifer
7 Sluyswacht
8 Cafe de Zon
9 Nossa Senhora
10 Maria
11 Oost-West
12 Little Thai Prince
13 San Francisco
14 caf√© De Pianist
15 De Dijk
16 Brasserie Flo
17 Porto Fino
